In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
def parse_latencies(target_dir):
    post_lats = []
    handle_lats = []

    proxy_log_path = next(iter(Path(target_dir).glob("**/http_proxy_*.log")))
    for line in proxy_log_path.read_text().splitlines():
        if "POST" in line:
            latency = float(line[line.rfind(" "):-2])
            post_lats.append(latency)
    print(len(post_lats))

    for replica_log_path in Path(target_dir).glob("**/deployment_classifier*.log"):
        for line in replica_log_path.read_text().splitlines():
            if "HANDLE" in line:
                latency = float(line[line.rfind(" "):-2])
                handle_lats.append(latency)
    print(len(handle_lats))
    return pd.DataFrame({"post": post_lats, "handle": handle_lats})

In [ ]:
target_dir = Path("/tmp/ray/session_latest/logs/serve")
df = parse_latencies(target_dir)

In [ ]:
print("p90", df.post.quantile(0.9))
print("average", df.post.mean())
print("median", df.post.median())
df.post.hist()

In [ ]:
print("p90", df.handle.quantile(0.9))
print("average", df.handle.mean())
print("median", df.handle.median())
df.handle.hist()

In [ ]:
overhead = df.post - df.handle
print("p90", overhead.quantile(0.9))
print("average", overhead.mean())
print("median", overhead.median())
print(overhead.hist())